# MongoDB 基本操作

In [1]:
import pymongo
from pymongo import MongoClient
from datetime import datetime

In [10]:
from pprint import pprint

In [12]:
from bson import ObjectId

### 连接mongo

In [2]:
# client = MongoClient()
# client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/') # URI
# 三种方法均可

In [53]:
class TestMongo():
    
    def __init__(self):
        self.client = MongoClient('mongodb://localhost:27017/')
        self.db = self.client['dingdian_novels']
    
    def add_one(self):
        ''' 插入数据 '''
        post = {
            'author': '天蚕土豆', 
            'words': '15000000',
            'update_time': str(datetime.now())
        }
        return self.db.dingdian_novels.insert_one(post)
    
    def get_one(self):
        ''' 查询一条数据 '''
        return self.db.dingdian_novels.find_one() # 返回的是字典类型
    
    def get_many(self):
        ''' 查询多条数据 '''
        return self.db.dingdian_novels.find({'author': '笑子风3'}) # 查询所有数据
    
    def get_one_from_oid(self, oid):
        ''' 通过id查询数据 '''
        oid = ObjectId(oid)
        return self.db.dingdian_novels.find({'_id': oid})
    
    def update_one(self):
        ''' 
        修改一条数据 
        .update_one 方法
        '''
        res = self.db.dingdian_novels.update_one({'author': '天蚕土豆'}, {'$inc': {'words': 10}})
        print('匹配数据条数:', res.matched_count)
        print('修改数据条数:', res.modified_count)
    
    def update_many(self):
        ''' 
        修改所有匹配数据 
        .update_many 方法
        '''
        res = self.db.dingdian_novels.update_many({'author': '天蚕土豆'}, {'$inc': {'words': 11}})
        print('匹配数据条数:', res.matched_count)
        print('修改数据条数:', res.modified_count)
    
    def delete_one(self):
        '''
        删除匹配到的第一条数据
        delete_one 方法
        '''
        res = self.db.dingdian_novels.delete_one({'author': '天蚕土豆'})
        print('删除数据条数:', res.deleted_count)
        
    def delete_many(self):
        '''
        删除所有匹配到的数据
        delete_many 方法
        '''
        res = self.db.dingdian_novels.delete_many({'author': '天蚕土豆'})
        print('删除数据条数:', res.deleted_count)

# MongoDB ODM
+ MongoEngine包

In [76]:
import mongoengine
from mongoengine import connect, Document
from mongoengine import StringField, IntField, FloatField
from mongoengine import EmbeddedDocument, ListField, EmbeddedDocumentField

In [69]:
connect('dingdian_novels', host='mongodb://localhost:27017/dingdian_novels')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary())

In [125]:
SEX_CHOICES = (
    ('MALE', '男'), 
    ('FEMALE', '女'),
)

class Grade(EmbeddedDocument):
    ''' 成绩 '''
    name = StringField(required=True)
    score = FloatField(required=True)

class Student(Document):
    name = StringField(max_length=32, required=True)
    age = IntField(required=True)
    sex = StringField(choices=SEX_CHOICES, required=True)
    grade = FloatField()
    address = StringField()
    grades = ListField(EmbeddedDocumentField(Grade))
    
    meta = {
        'collection': 'students666', # 制定存放的collection的名称
        'ordering': ['-age'], # 以年纪倒叙
    }

In [131]:
class TestMongoEngine():
    
    def add_one(self):
        ''' 添加数据 '''
        
        yuwen = Grade(
            name = '语文', 
            score = 90
        )
        shuxue = Grade(
            name = '数学', 
            score = 100
        )
        stu_obj = Student(
            name = '张宇东',
            age = 155, 
            sex = 'MALE',
            grades = [yuwen, shuxue],
        )
        stu_obj.save()
        return stu_obj
    
    def get_one(self):
        ''' 查询一条数据 '''
        return Student.objects.first()
    
    def get_all(self):
        ''' 查询多条数据 '''
        return Student.objects.all()
    
    def get_from_oid(self, oid):
        ''' 根据ID来获取数据 '''
        return Student.objects.filter(pk=oid).first()
    
    def update_one(self):
        ''' 修改一条数据 '''
        res = Student.objects.filter(age=155).update_one(inc__age=1)
        return res
    
    def update_many(self):
        ''' 修改所有匹配到的数据 '''
        res = Student.objects.filter(age=15).update(inc__age=1000)
        return res
    
    def delete_one(self):
        ''' 删除一条数据 '''
        res = Student.objects.filter(age=1015).first().delete()
    
    def delete_many(self):
        ''' 删除所有匹配的数据 '''
        res = Student.objects.filter(age=1015).delete()

In [134]:
obj = TestMongoEngine()
obj.delete_many()

In [116]:
res.id

ObjectId('5aedd94204e83c4a59ed9c43')